In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head(10)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [3]:
# Look at APPLICATION_TYPE value counts for binning
app_vcounts = application_df["APPLICATION_TYPE"].value_counts()
app_vcounts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_vcounts[app_vcounts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
class_vcounts = application_df["CLASSIFICATION"].value_counts()
class_vcounts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_vcounts[class_vcounts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cats = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cats]))


# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cats)
encode_df.head()

c:\Users\Larfy\anaconda3\envs\mlenv2\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cats, 1)
application_df.head()

C:\Users\Larfy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(["IS_SUCCESSFUL"], 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

C:\Users\Larfy\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 9,501
Trainable params: 9,501
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_opt/",exist_ok=True)
checkpoint_path = "checkpoints_opt/weights.{epoch:02d}.hdf5"

In [14]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5709 - accuracy: 0.7238
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5555 - accuracy: 0.7292
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5521 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5502 - accuracy: 0.7311
Epoch 5/100
800/804 [============================>.] - ETA: 0s - loss: 0.5497 - accuracy: 0.7314
Epoch 5: saving model to checkpoints_opt\weights.05.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5495 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5473 - accuracy: 0.7323
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5468 - accuracy: 0.7343
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5460 - accuracy: 0.7336
Epoch 9/100
804/804 [=============================

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6378 - accuracy: 0.7250 - 889ms/epoch - 3ms/step
Loss: 0.6377671360969543, Accuracy: 0.7250145673751831


In [16]:
nn.save("AlphabetSoupCharity_Optimization.h5")

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features2 = len(X_train_scaled[0])
hidden_nodes_2layer1 = 200
hidden_nodes_2layer2 = 100

nn_two = tf.keras.models.Sequential()

# First hidden layer
nn_two.add(
    tf.keras.layers.Dense(units=hidden_nodes_2layer1, input_dim=number_input_features2, activation="relu")
)

# Second hidden layer
nn_two.add(tf.keras.layers.Dense(units=hidden_nodes_2layer2, activation="tanh"))

# Output layer
nn_two.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_two.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 200)               8800      
                                                                 
 dense_4 (Dense)             (None, 100)               20100     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 29,001
Trainable params: 29,001
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn_two.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints_opttwo/",exist_ok=True)
checkpoint_path = "checkpoints_opttwo/weights.{epoch:02d}.hdf5"

In [19]:
# Create a callback that saves the model's weights every epoch
cp_callback2 = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn_two.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback2])

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5700 - accuracy: 0.7213
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5561 - accuracy: 0.7288
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5533 - accuracy: 0.7308
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5510 - accuracy: 0.7306
Epoch 5/100
803/804 [============================>.] - ETA: 0s - loss: 0.5500 - accuracy: 0.7317
Epoch 5: saving model to checkpoints_opttwo\weights.05.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5499 - accuracy: 0.7317
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5491 - accuracy: 0.7322
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5480 - accuracy: 0.7329
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5475 - accuracy: 0.7324
Epoch 9/100
804/804 [==========================

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_two.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5641 - accuracy: 0.7249 - 960ms/epoch - 4ms/step
Loss: 0.5640577077865601, Accuracy: 0.7248979806900024


In [21]:
nn_two.save("AlphabetSoupCharity_Optimization.h5")

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features3 = len(X_train_scaled[0])
hidden_nodes_3layer1 = 8
hidden_nodes_3layer2 = 5
hidden_nodes_3layer3 = 2

nn_three = tf.keras.models.Sequential()

# First hidden layer
nn_three.add(
    tf.keras.layers.Dense(units=hidden_nodes_3layer1, input_dim=number_input_features3, activation="tanh")
)

# Second hidden layer
nn_three.add(tf.keras.layers.Dense(units=hidden_nodes_3layer2, activation="relu"))

# Third hidden layer
nn_three.add(tf.keras.layers.Dense(units=hidden_nodes_3layer3, activation="relu"))

# Output layer
nn_three.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_three.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 8)                 352       
                                                                 
 dense_7 (Dense)             (None, 5)                 45        
                                                                 
 dense_8 (Dense)             (None, 2)                 12        
                                                                 
 dense_9 (Dense)             (None, 1)                 3         
                                                                 
Total params: 412
Trainable params: 412
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn_three.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints_optthree/",exist_ok=True)
checkpoint_path = "checkpoints_optthree/weights.{epoch:02d}.hdf5"

In [24]:
# Create a callback that saves the model's weights every epoch
cp_callback3 = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn_three.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback3])

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6314 - accuracy: 0.6672
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5836 - accuracy: 0.7158
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5709 - accuracy: 0.7229
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5617 - accuracy: 0.7294
Epoch 5/100
793/804 [============================>.] - ETA: 0s - loss: 0.5591 - accuracy: 0.7309
Epoch 5: saving model to checkpoints_optthree\weights.05.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5588 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5577 - accuracy: 0.7311
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5568 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5566 - accuracy: 0.7318
Epoch 9/100
804/804 [========================

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_three.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5651 - accuracy: 0.7228 - 654ms/epoch - 2ms/step
Loss: 0.5651419758796692, Accuracy: 0.7227988243103027


In [26]:
nn_three.save("AlphabetSoupCharity_Optimization.h5")